In [1]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Data

In [2]:
# features = np.genfromtxt('train_features.csv', delimiter=',')[1:]
# labels = np.genfromtxt('train_labels.csv', delimiter=',')[1:]

In [3]:
features = pd.read_csv('train_features.csv')
labels = pd.read_csv('train_labels.csv')

In [4]:
features

,pid,Time,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,3,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,4,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,5,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,6,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,7,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227935,9999,8,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,80.0,NaN,NaN,110.0,NaN
227936,9999,9,85.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,83.0,NaN,NaN,123.0,NaN
227937,9999,10,85.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,...,NaN,98.0,NaN,NaN,NaN,80.0,NaN,NaN,138.0,NaN
227938,9999,11,85.0,NaN,NaN,NaN,NaN,NaN,10.2,NaN,...,NaN,98.0,NaN,NaN,31.0,75.0,NaN,NaN,125.0,NaN


In [5]:
labels

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18990,9993,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,17.1,69.8,100.0,110.7
18991,9995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.6,97.3,97.8,59.2
18992,9996,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,17.3,66.3,96.9,100.3
18993,9998,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.8,81.5,96.9,99.4


In [6]:
labels.astype(bool).sum(axis=0)

pid                       18995
LABEL_BaseExcess           5096
LABEL_Fibrinogen           1400
LABEL_AST                  4554
LABEL_Alkalinephos         4487
LABEL_Bilirubin_total      4570
LABEL_Lactate              3803
LABEL_TroponinI            1895
LABEL_SaO2                 4439
LABEL_Bilirubin_direct      644
LABEL_EtCO2                1254
LABEL_Sepsis               1088
LABEL_RRate               18995
LABEL_ABPm                18995
LABEL_SpO2                18995
LABEL_Heartrate           18995
dtype: int64

# Preprocessing

In [7]:
# Sort features and remove time
labels.sort_values('pid', inplace=True)

features.sort_values('Time', inplace=True)
features.sort_values('pid', kind='stable', inplace=True)
features.drop('Time', axis='columns', inplace=True)
features

,pid,Age,EtCO2,PTT,BUN,Lactate,Temp,Hgb,HCO3,BaseExcess,...,Alkalinephos,SpO2,Bilirubin_direct,Chloride,Hct,Heartrate,Bilirubin_total,TroponinI,ABPs,pH
0,1,34.0,NaN,NaN,12.0,NaN,36.0,8.7,24.0,-2.0,...,NaN,100.0,NaN,114.0,24.6,94.0,NaN,NaN,142.0,7.33
1,1,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,-2.0,...,NaN,100.0,NaN,NaN,NaN,99.0,NaN,NaN,125.0,7.33
2,1,34.0,NaN,NaN,NaN,NaN,36.0,NaN,NaN,0.0,...,NaN,100.0,NaN,NaN,NaN,92.0,NaN,NaN,110.0,7.37
3,1,34.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,0.0,...,NaN,100.0,NaN,NaN,NaN,88.0,NaN,NaN,104.0,7.37
4,1,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,100.0,NaN,NaN,22.4,81.0,NaN,NaN,100.0,7.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172879,31658,60.0,NaN,NaN,NaN,NaN,37.0,NaN,NaN,NaN,...,NaN,96.0,NaN,NaN,NaN,71.0,NaN,NaN,127.0,NaN
172880,31658,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172881,31658,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172882,31658,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,96.0,NaN,NaN,NaN,71.0,NaN,NaN,135.0,NaN


In [8]:
features_mean = features.groupby('pid').mean().add_suffix('_mean')
features_std = features.groupby('pid').std().add_suffix('_std')
# features_count = features.groupby('pid').count().add_suffix('_count')

In [9]:
full_features = pd.concat([features_mean, features_std], axis='columns')
full_features

,Age_mean,EtCO2_mean,PTT_mean,BUN_mean,Lactate_mean,Temp_mean,Hgb_mean,HCO3_mean,BaseExcess_mean,RRate_mean,...,Alkalinephos_std,SpO2_std,Bilirubin_direct_std,Chloride_std,Hct_std,Heartrate_std,Bilirubin_total_std,TroponinI_std,ABPs_std,pH_std
pid,,,,,,,,,,,,,,,,,,,,,
1,34.0,NaN,NaN,12.0,NaN,36.750000,8.566667,25.333333,-0.666667,17.000000,...,NaN,0.000000,NaN,1.732051,0.995992,13.918322,NaN,NaN,12.086807,0.030551
2,86.0,NaN,31.8,32.0,NaN,36.000000,13.100000,NaN,NaN,18.000000,...,NaN,1.414214,NaN,NaN,NaN,0.000000,NaN,NaN,8.530160,NaN
4,66.0,NaN,34.6,8.0,NaN,36.666667,10.550000,NaN,NaN,14.636364,...,NaN,1.420627,NaN,NaN,2.192031,3.357488,NaN,NaN,31.010849,NaN
6,66.0,NaN,53.8,32.0,1.8,37.166667,10.316667,19.500000,-2.857143,15.833333,...,NaN,1.302678,NaN,3.535534,1.298247,3.339388,NaN,NaN,5.773503,0.020587
8,42.0,NaN,NaN,18.0,NaN,36.000000,NaN,NaN,NaN,17.181818,...,NaN,1.932184,NaN,NaN,NaN,8.109478,NaN,NaN,27.361885,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31653,52.0,NaN,25.8,11.0,1.7,36.000000,9.500000,22.666667,0.000000,15.545455,...,NaN,1.328020,NaN,0.000000,1.880957,15.172942,NaN,NaN,7.474401,0.000000
31654,66.0,NaN,NaN,33.0,NaN,37.500000,11.200000,NaN,NaN,16.909091,...,NaN,1.578261,NaN,NaN,NaN,12.028753,NaN,NaN,11.180340,NaN
31656,44.0,NaN,NaN,15.0,NaN,38.000000,12.400000,24.000000,-3.500000,24.900000,...,NaN,0.000000,NaN,NaN,NaN,6.983313,NaN,NaN,11.106054,0.042426


In [10]:
full_features.fillna(full_features.mean(axis='rows'), inplace=True)

In [11]:
X = StandardScaler().fit_transform(full_features)
X

array([[-1.70642596,  0.        ,  0.        , ...,  0.        ,
        -0.03665977, -0.43335258],
       [ 1.454319  ,  0.        , -0.45881356, ...,  0.        ,
        -0.66160797,  0.        ],
       [ 0.23864786,  0.        , -0.27532595, ...,  0.        ,
         3.28853559,  0.        ],
       ...,
       [-1.09859039,  0.        ,  0.        , ...,  0.        ,
        -0.20899055,  0.36213798],
       [ 0.48178209,  0.        ,  0.        , ...,  0.        ,
        -0.94238798,  0.        ],
       [-0.12605348,  0.        ,  0.        , ...,  0.        ,
        -1.3288414 ,  0.        ]])

In [12]:
np.nanmean(X, axis=0), np.nanstd(X, axis=0)

(array([-1.05767812e-16,  5.98509280e-17,  5.04992205e-17,  1.87034150e-18,
        -1.00100677e-15, -4.30178545e-15,  1.44390364e-16, -1.61728429e-15,
        -4.30178545e-18,  1.61597506e-16,  1.46260705e-15, -5.06020893e-16,
        -6.69582257e-17, -9.42652116e-17, -8.11915245e-16, -8.97763920e-17,
        -1.34664588e-17, -3.29928241e-16,  3.49080538e-15,  4.79555561e-16,
         1.57108686e-17,  4.59355872e-16, -7.44395917e-16,  1.48056233e-15,
        -1.55425379e-16,  2.95513957e-16,  4.84792517e-16,  4.50565267e-16,
         1.54677242e-15, -6.39282725e-16, -2.59229332e-16,  3.25439421e-17,
         6.88285672e-17,  1.38405271e-17, -2.00869066e-14,  0.00000000e+00,
         7.49258805e-16,  2.86583076e-16, -1.93393311e-16,  3.07390625e-16,
        -1.23442539e-17, -3.02995323e-16, -3.88843998e-16,  6.65841574e-17,
         1.81049057e-16,  8.97763920e-18, -7.14096385e-16, -1.15961173e-17,
        -6.50878842e-17, -4.24193452e-16,  2.01996882e-16, -6.73322940e-18,
         3.0

# Prep

In [19]:
grouped_features = np.array(np.array_split(features, np.flatnonzero(np.diff(features[:, 0])) + 1))
np.array(grouped_features).shape, features.shape

((18995, 12, 37), (227940, 37))

In [20]:
grouped_features = np.delete(grouped_features, 0, 2) # remove pid
grouped_features = np.delete(grouped_features, 0, 2) # remove time

In [21]:
def interpolate(arr: np.ndarray) -> np.ndarray:
  ok = ~np.isnan(arr)
  xp = ok.ravel().nonzero()[0]
  fp = arr[~np.isnan(arr)]
  x  = np.isnan(arr).ravel().nonzero()[0]
  if xp.shape[0] > 0:
    arr[np.isnan(arr)] = np.interp(x, xp, fp)

for i in range(grouped_features.shape[0]):
  patient = grouped_features[i]
  np.apply_along_axis(interpolate, 0, patient)


In [22]:
mean_baseline = np.nanmean(grouped_features, axis=(0,1))
std_baseline = np.nanstd(grouped_features, axis=(0,1))
mean_baseline, std_baseline

(array([ 62.0738089 ,  32.89301248,  38.6570739 ,  23.00099348,
          2.34876847,  36.76950149,  10.75408998,  23.80565001,
         -0.76196829,  18.11960675, 273.37989839,   3.61664545,
         11.5668856 ,   1.47022377,  40.82189725, 158.22970097,
          0.65844312, 210.72741735,  93.27103528, 138.97713623,
         82.58048099,   1.99608957,   4.11529034,  64.51494365,
          7.74215739,  96.6238099 ,  97.61809765,   1.17152755,
        105.92167854,  32.02225084,  84.39287094,   1.54361645,
          6.45230704, 122.85433358,   7.3768517 ]),
 array([1.64518177e+01, 9.31588410e+00, 2.44412744e+01, 1.98353224e+01,
        1.88923081e+00, 8.02167052e-01, 2.08755924e+00, 4.26810800e+00,
        4.23162575e+00, 5.09278376e+00, 1.39269867e+02, 1.32330290e+00,
        9.23669198e+00, 1.89784478e+00, 9.12986454e+00, 6.03430712e+02,
        2.10639450e+01, 1.03497854e+02, 1.02485062e+01, 5.36626447e+01,
        1.65846626e+01, 4.06501020e-01, 6.32774843e-01, 1.40699296e+01,
    

In [23]:
X = np.empty((grouped_features.shape[0], 2 * grouped_features.shape[2]))

for i in range(grouped_features.shape[0]):
  patient = grouped_features[i]

  mean = np.nanmean(patient, axis=0)
  std = np.nanstd(patient, axis=0)

  mean[np.isnan(mean)] = mean_baseline[np.isnan(mean)]
  std[np.isnan(std)] = std_baseline[np.isnan(std)]

  X[i] = np.hstack((mean, std))


/tmp/ipykernel_1044/50045206.py:6: RuntimeWarning: Mean of empty slice
  mean = np.nanmean(patient, axis=0)
/home/jonhue/.local/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [10]:
# def interpolate(arr: np.ndarray) -> np.ndarray:
#   ok = ~np.isnan(arr)
#   xp = ok.ravel().nonzero()[0]
#   fp = arr[~np.isnan(arr)]
#   x  = np.isnan(arr).ravel().nonzero()[0]
#   if xp.shape[0] > 0:
#     arr[np.isnan(arr)] = np.interp(x, xp, fp)
#   # return np.zeros_like(arr)

# def diff(arr: np.ndarray) -> np.ndarray:
#   # print(arr, np.zeros_like(arr))
#   # print(arr, np.insert(np.diff(arr), 0, np.nan))
#   arr[1:] = np.diff(arr)
#   arr[0] = np.nan
#   # return np.insert(np.diff(arr), 0, np.nan)
#   # return np.ones_like(arr)

# grouped_features_ext = np.empty((grouped_features.shape[0], grouped_features.shape[1], 2 * grouped_features.shape[2]))

# for i in range(grouped_features.shape[0]):
#   patient = grouped_features[i]
#   # patient[:,TIME] = patient[:,TIME] - patient[0,TIME]
#   np.apply_along_axis(interpolate, 0, patient)

#   change_patient = patient.copy()
#   np.apply_along_axis(diff, 0, change_patient)

#   grouped_features_ext[i] = np.hstack((patient, change_patient))


# Task 1

In [13]:
y = labels['LABEL_Sepsis']
y

0        0.0
6622     0.0
15008    0.0
16335    0.0
17676    0.0
        ... 
14402    0.0
14403    0.0
14404    0.0
14405    0.0
14406    0.0
Name: LABEL_Sepsis, Length: 18995, dtype: float64

In [14]:
clf = SVC(probability=True, class_weight='balanced')
clf.fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

# Analysis

In [14]:
import sklearn.metrics as metrics

In [ ]:
y_pred = clf.predict_proba(X)[:,1]
y_pred

In [18]:
y_pred

array([0.05955863, 0.05957678, 0.05953758, ..., 0.05959573, 0.05957092,
       0.0596306 ])

In [16]:
metrics.accuracy_score(y, clf.predict(X))

0.9427217688865491

In [21]:
metrics.recall_score(y, clf.predict(X))

0.5174632352941176

In [23]:
metrics.precision_score(y, clf.predict(X))

In [ ]:
metrics.roc_auc_score(y, y_pred)

0.3228822260601342

In [89]:
from score_submission import get_score

In [90]:
import pandas as pd

In [91]:
true = labels.iloc[0:20]
true

,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,0.479470,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.068871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,0.461657,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.353318,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.809785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
5,10002,0.327519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
6,10006,0.094427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
7,10007,0.006737,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
8,10009,0.267901,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5
9,1001,0.680320,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.9,83.3,99.9,117.3


In [92]:
submission = true
# submission['pid'] = true['pid']
# submission = submission.rename(columns={0: 'LABEL_BaseExcess'})
submission['LABEL_BaseExcess'] = final_pred
submission

/home/jonhue/.local/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


,pid,LABEL_BaseExcess,LABEL_Fibrinogen,LABEL_AST,LABEL_Alkalinephos,LABEL_Bilirubin_total,LABEL_Lactate,LABEL_TroponinI,LABEL_SaO2,LABEL_Bilirubin_direct,LABEL_EtCO2,LABEL_Sepsis,LABEL_RRate,LABEL_ABPm,LABEL_SpO2,LABEL_Heartrate
0,1,0.479470,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,12.1,85.4,100.0,59.9
1,10,0.068871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.8,100.6,95.5,85.5
2,100,0.461657,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,16.5,88.3,96.5,108.1
3,1000,0.353318,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,19.4,77.2,98.3,80.9
4,10000,0.809785,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.6,76.8,97.7,95.3
5,10002,0.327519,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.5,67.4,99.1,64.6
6,10006,0.094427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.2,84.9,96.8,90.5
7,10007,0.006737,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,23.8,97.0,94.3,76.0
8,10009,0.267901,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,18.9,72.0,95.3,91.5
9,1001,0.680320,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,14.9,83.3,99.9,117.3


In [93]:
get_score(true, submission)

ValueError: continuous format is not supported